# float64

In [3]:
import math
import numpy as np
from numba import cuda

tol = 1e-15
max_terms = 4096

@cuda.jit(device=True)
def _2F1_device(a, b, c, z_r, z_i):
    if c == 0.0:
        return 1.0, 0.0
    real_accum = 1.0
    imag_accum = 0.0
    term_r = 1.0
    term_i = 0.0
    for n in range(1, max_terms):
        denom = n*(c+n-1.0)
        poch = ((a+n-1.0)*(b+n-1.0))/denom
        zr = term_r*z_r - term_i*z_i
        zi = term_r*z_i + term_i*z_r
        term_r = poch*zr
        term_i = poch*zi
        if (abs(term_r)<tol) and (abs(term_i)<tol):
            break
        real_accum += term_r
        imag_accum += term_i
    return real_accum, imag_accum

@cuda.jit(device=True)
def compute_g_device(d_val, s_val, x, y):
    h = 0.5*(d_val + s_val)
    hb = 0.5*(d_val - s_val)
    d = h+hb
    s = h-hb

    r = math.sqrt(x*x + y*y)
    theta = math.atan2(y,x)

    r_pow_d = r**d
    cos_st = math.cos(s*theta)
    sin_st = math.sin(s*theta)
    z_plus_r = r_pow_d*cos_st
    z_plus_i = r_pow_d*sin_st

    z_minus_r = r_pow_d*cos_st
    z_minus_i = -r_pow_d*sin_st

    z_r, z_i = x, y
    z_b_r, z_b_i = x, -y

    fhz_r, fhz_i = _2F1_device(h,h,2*h,z_r,z_i)
    fhbz_b_r, fhbz_b_i = _2F1_device(hb,hb,2*hb,z_b_r,z_b_i)
    fhz_b_r, fhz_b_i = _2F1_device(h,h,2*h,z_b_r,z_b_i)
    fhb_z_r, fhb_z_i = _2F1_device(hb,hb,2*hb,z_r,z_i)

    # T1 = z^h z_b^{h_b} * fhz * fhbz_b
    step1_r = fhz_r*fhbz_b_r - fhz_i*fhbz_b_i
    step1_i = fhz_r*fhbz_b_i + fhz_i*fhbz_b_r
    T1_r = step1_r*z_plus_r - step1_i*z_plus_i
    T1_i = step1_r*z_plus_i + step1_i*z_plus_r

    # T2 = z_b^h z^{h_b} * fhz_b * fhb_z
    step2_r = fhz_b_r*fhb_z_r - fhz_b_i*fhb_z_i
    step2_i = fhz_b_r*fhb_z_i + fhz_b_i*fhb_z_r
    T2_r = step2_r*z_minus_r - step2_i*z_minus_i
    T2_i = step2_r*z_minus_i + step2_i*z_minus_r

    g_r = T1_r + T2_r
    g_i = T1_i + T2_i
    return g_r, g_i

@cuda.jit
def compute_g_delta_kernel(d_arr, s_arr, x_arr, y_arr, dphi, g_delta_matrix):
    j, i = cuda.grid(2)
    N = x_arr.size
    M = d_arr.size
    if j < N and i < M:
        d_val = d_arr[i]
        s_val = s_arr[i]

        x = x_arr[j]
        y = y_arr[j]
        g1_r, g1_i = compute_g_device(d_val, s_val, x, y)

        xp = 1.0 - x
        yp = -y
        g2_r, g2_i = compute_g_device(d_val, s_val, xp, yp)

        zm1_r = x - 1.0
        zm1_i = y
        zbm1_r = x - 1.0
        zbm1_i = -y
        prod1_r = zm1_r*zbm1_r - zm1_i*zbm1_i
        prod1_i = zm1_r*zbm1_i + zm1_i*zbm1_r
        r1 = math.sqrt(prod1_r*prod1_r + prod1_i*prod1_i)
        theta1 = math.atan2(prod1_i, prod1_r)

        zzb = x*x + y*y
        r1_pow = r1**dphi
        cos_th1 = math.cos(dphi*theta1)
        sin_th1 = math.sin(dphi*theta1)
        factor1_r = r1_pow*cos_th1
        factor1_i = r1_pow*sin_th1
        r2_pow = zzb**dphi

        A_r = factor1_r*g1_r - factor1_i*g1_i
        A_i = factor1_r*g1_i + factor1_i*g1_r
        B_r = r2_pow*g2_r
        B_i = r2_pow*g2_i

        gdelta_r = A_r - B_r
        # gdelta is real
        idx = j*M + i
        g_delta_matrix[idx] = gdelta_r

def calculate_g_delta_matrix(d_values, s_values, x_values, y_values, dphi):
    N = len(x_values)
    M = len(d_values)
    g_delta_device = cuda.device_array(N*M, dtype=np.float64)

    # Transfer data
    d_device = cuda.to_device(np.array(d_values,dtype=np.float64))
    s_device = cuda.to_device(np.array(s_values,dtype=np.float64))
    x_device = cuda.to_device(x_values.astype(np.float64))
    y_device = cuda.to_device(y_values.astype(np.float64))

    threads_per_block = (16,16)
    blocks_x = (M+threads_per_block[0]-1)//threads_per_block[0]
    blocks_y = (N+threads_per_block[1]-1)//threads_per_block[1]

    compute_g_delta_kernel[(blocks_y,blocks_x), threads_per_block](
        d_device, s_device, x_device, y_device, dphi, g_delta_device
    )

    return g_delta_device.copy_to_host().reshape(N,M)


In [4]:
# Example usage:

# 4 (d,s) pairs
d_values = [1.0, 1.5, 2.0, 2.5]
s_values = [0.5, 0.75, 1.0, 1.25]

# 1000 z's
N=1000
x_values = np.linspace(0.01,0.5,N)
y_values = np.linspace(0.01,0.5,N)

dphi = 0.3

%timeit calculate_g_delta_matrix(d_values, s_values, x_values, y_values, dphi)

g_delta = calculate_g_delta_matrix(d_values, s_values, x_values, y_values, dphi)
print("g_delta matrix (float64, tol=1e-20, max_terms=2048):")
print(g_delta)


/home/string-0/anaconda3/envs/RL/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 63 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


9.35 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
g_delta matrix (float64, tol=1e-20, max_terms=2048):
[[-7.31952693e-01 -1.47023054e+00 -2.78928263e+00 -5.20283284e+00]
 [-7.42409215e-01 -1.48641673e+00 -2.80942804e+00 -5.22149865e+00]
 [-7.52418680e-01 -1.50177069e+00 -2.82813953e+00 -5.23783465e+00]
 ...
 [-2.57251746e-03 -3.69346898e-03 -3.73415618e-03 -2.68720800e-03]
 [-1.28593956e-03 -1.84611001e-03 -1.86582625e-03 -1.34123135e-03]
 [ 4.65239062e-18  4.71542708e-17  1.37402683e-17  4.78155017e-18]]


# float128

In [1]:
import math
import numpy as np
from numba import cuda

tol = 1e-20
max_terms = 2048
precision_mode = 1  # double-double (float128 emulation)

@cuda.jit(device=True,inline=True)
def dd_add(a_hi, a_lo, b_hi, b_lo):
    s = a_hi + b_hi
    v = s - a_hi
    t = (b_hi - v) + (a_hi - (s - v))
    e = a_lo + b_lo + t
    hi = s + e
    lo = e - (hi - s)
    return hi, lo

@cuda.jit(device=True,inline=True)
def dd_mul(a_hi, a_lo, b_hi, b_lo):
    p = a_hi * b_hi
    cross = a_hi*b_lo + a_lo*b_hi
    approx_hi = p + cross
    approx_lo = (p - approx_hi) + cross + (a_lo*b_lo)
    hi = approx_hi + approx_lo
    lo = approx_lo - (hi - approx_hi)
    return hi, lo

@cuda.jit(device=True,inline=True)
def dd_add_double(a_hi, a_lo, x):
    s = a_hi + x
    v = s - a_hi
    t = (x - v) + (a_hi - (s - v))
    e = a_lo + t
    hi = s + e
    lo = e - (hi - s)
    return hi, lo

@cuda.jit(device=True,inline=True)
def dd_abs(a_hi, a_lo):
    if a_hi < 0.0 or (a_hi == 0.0 and a_lo < 0.0):
        return -a_hi, -a_lo
    else:
        return a_hi, a_lo

@cuda.jit(device=True,inline=True)
def dd_less(a_hi, a_lo, x):
    ah, al = dd_abs(a_hi, a_lo)
    if ah < x or (ah == x and al < 0.0):
        return True
    return False

@cuda.jit(device=True)
def _2F1_device(a, b, c, z_r, z_i, tol, max_terms, precision_mode):
    if precision_mode == 0:
        # float64 mode (not used here but kept for completeness)
        if c == 0.0:
            return 1.0,0.0
        real_accum=1.0
        imag_accum=0.0
        term_r=1.0
        term_i=0.0
        for n in range(1,max_terms):
            denom = n*(c+n-1.0)
            poch=((a+n-1.0)*(b+n-1.0))/denom
            zr=term_r*z_r - term_i*z_i
            zi=term_r*z_i+term_i*z_r
            term_r=poch*zr
            term_i=poch*zi
            if (abs(term_r)<tol) and (abs(term_i)<tol):
                break
            real_accum+=term_r
            imag_accum+=term_i
        return real_accum, imag_accum
    else:
        # double-double mode
        if c == 0.0:
            return 1.0,0.0

        # double-double accumulators
        real_accum_hi = 1.0
        real_accum_lo = 0.0
        imag_accum_hi = 0.0
        imag_accum_lo = 0.0

        term_r_hi=1.0
        term_r_lo=0.0
        term_i_hi=0.0
        term_i_lo=0.0

        local_tol = tol if tol>1e-15 else 1e-15

        for n in range(1,max_terms):
            denom = n*(c+n-1.0)
            poch=((a+n-1.0)*(b+n-1.0))/denom
            poch_hi=poch
            poch_lo=0.0

            # multiply term by z approximately (no dd for z)
            zr_hi=term_r_hi*z_r - term_i_hi*z_i
            zr_lo=term_r_lo*z_r - term_i_lo*z_i
            zi_hi=term_r_hi*z_i + term_i_hi*z_r
            zi_lo=term_r_lo*z_i + term_i_lo*z_r

            nr_hi,nr_lo=dd_mul(poch_hi,poch_lo,zr_hi,zr_lo)
            ni_hi,ni_lo=dd_mul(poch_hi,poch_lo,zi_hi,zi_lo)

            term_r_hi,term_r_lo=nr_hi,nr_lo
            term_i_hi,term_i_lo=ni_hi,ni_lo

            ar_hi,ar_lo=dd_abs(term_r_hi,term_r_lo)
            ai_hi,ai_lo=dd_abs(term_i_hi,term_i_lo)

            if dd_less(ar_hi,ar_lo,local_tol) and dd_less(ai_hi,ai_lo,local_tol):
                break

            # accumulate real
            real_accum_hi,real_accum_lo = dd_add(real_accum_hi,real_accum_lo,term_r_hi,term_r_lo)
            # accumulate imag
            imag_accum_hi,imag_accum_lo = dd_add(imag_accum_hi,imag_accum_lo,term_i_hi,term_i_lo)

        real_val = real_accum_hi+real_accum_lo
        imag_val = imag_accum_hi+imag_accum_lo
        return real_val, imag_val

@cuda.jit(device=True)
def compute_g_device(d_val, s_val, x, y, tol, max_terms, precision_mode):
    h=0.5*(d_val+s_val)
    hb=0.5*(d_val-s_val)
    d=h+hb
    s=h-hb
    r=math.sqrt(x*x+y*y)
    theta=math.atan2(y,x)

    r_pow_d=r**d
    cos_st=math.cos(s*theta)
    sin_st=math.sin(s*theta)
    z_plus_r=r_pow_d*cos_st
    z_plus_i=r_pow_d*sin_st

    z_minus_r=r_pow_d*cos_st
    z_minus_i=-r_pow_d*sin_st

    z_r=x
    z_i=y
    z_b_r=x
    z_b_i=-y

    fhz_r,fhz_i=_2F1_device(h,h,2*h,z_r,z_i,tol,max_terms,precision_mode)
    fhbz_b_r,fhbz_b_i=_2F1_device(hb,hb,2*hb,z_b_r,z_b_i,tol,max_terms,precision_mode)
    fhz_b_r,fhz_b_i=_2F1_device(h,h,2*h,z_b_r,z_b_i,tol,max_terms,precision_mode)
    fhb_z_r,fhb_z_i=_2F1_device(hb,hb,2*hb,z_r,z_i,tol,max_terms,precision_mode)

    step1_r=fhz_r*fhbz_b_r - fhz_i*fhbz_b_i
    step1_i=fhz_r*fhbz_b_i + fhz_i*fhbz_b_r
    T1_r=step1_r*z_plus_r - step1_i*z_plus_i
    T1_i=step1_r*z_plus_i + step1_i*z_plus_r

    step2_r=fhz_b_r*fhb_z_r - fhz_b_i*fhb_z_i
    step2_i=fhz_b_r*fhb_z_i + fhz_b_i*fhb_z_r
    T2_r=step2_r*z_minus_r - step2_i*z_minus_i
    T2_i=step2_r*z_minus_i + step2_i*z_minus_r

    g_r=T1_r+T2_r
    g_i=T1_i+T2_i
    return g_r,g_i

@cuda.jit
def compute_g_delta_kernel(d_arr, s_arr, x_arr, y_arr, dphi, g_delta_matrix):
    j,i = cuda.grid(2)
    N=x_arr.size
    M=d_arr.size
    if j<N and i<M:
        d_val=d_arr[i]
        s_val=s_arr[i]
        x=x_arr[j]
        y=y_arr[j]

        g1_r,g1_i=compute_g_device(d_val,s_val,x,y,tol,max_terms,precision_mode)
        xp=1.0 - x
        yp=-y
        g2_r,g2_i=compute_g_device(d_val,s_val,xp,yp,tol,max_terms,precision_mode)

        zm1_r=x-1.0
        zm1_i=y
        zbm1_r=x-1.0
        zbm1_i=-y
        prod1_r=zm1_r*zbm1_r - zm1_i*zbm1_i
        prod1_i=zm1_r*zbm1_i + zm1_i*zbm1_r
        r1=math.sqrt(prod1_r*prod1_r+prod1_i*prod1_i)
        theta1=math.atan2(prod1_i,prod1_r)

        zzb=x*x+y*y
        r1_pow=r1**dphi
        cos_th1=math.cos(dphi*theta1)
        sin_th1=math.sin(dphi*theta1)
        factor1_r=r1_pow*cos_th1
        factor1_i=r1_pow*sin_th1
        r2_pow=(zzb**dphi)

        A_r=factor1_r*g1_r - factor1_i*g1_i
        A_i=factor1_r*g1_i + factor1_i*g1_r
        B_r=r2_pow*g2_r
        B_i=r2_pow*g2_i

        gdelta_r=A_r - B_r
        # g_delta is real
        idx=j*M+i
        g_delta_matrix[idx]=gdelta_r

def calculate_g_delta_matrix(d_values, s_values, x_values, y_values, dphi):
    N=len(x_values)
    M=len(d_values)

    g_delta_device=cuda.device_array(N*M,dtype=np.float64)
    d_device=cuda.to_device(np.array(d_values,dtype=np.float64))
    s_device=cuda.to_device(np.array(s_values,dtype=np.float64))
    x_device=cuda.to_device(x_values.astype(np.float64))
    y_device=cuda.to_device(y_values.astype(np.float64))

    threads_per_block=(16,16)
    blocks_x=(M+threads_per_block[0]-1)//threads_per_block[0]
    blocks_y=(N+threads_per_block[1]-1)//threads_per_block[1]

    compute_g_delta_kernel[(blocks_y,blocks_x),threads_per_block](d_device,s_device,x_device,y_device,dphi,g_delta_device)

    return g_delta_device.copy_to_host().reshape(N,M)


In [2]:
# Example usage:

d_values=[1.0,1.5,2.0,2.5]
s_values=[0.5,0.75,1.0,1.25]
N=100
x_values=np.linspace(0.01,0.5,N)
y_values=np.linspace(0.01,0.5,N)
dphi=0.3

%timeit calculate_g_delta_matrix(d_values, s_values, x_values, y_values, dphi)

g_delta_matrix=calculate_g_delta_matrix(d_values, s_values, x_values, y_values, dphi)
print(f"g_delta matrix (float128-like, tol={tol}, max_terms={max_terms}):")
print(g_delta_matrix)

/home/string-0/anaconda3/envs/RL/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 7 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


18.7 ms ± 500 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
g_delta matrix (float128-like, tol=1e-20, max_terms=2048):
[[-7.31952693e-01 -1.47023054e+00 -2.78928263e+00 -5.20283284e+00]
 [-8.20353722e-01 -1.60210870e+00 -2.93933054e+00 -5.30672294e+00]
 [-8.82566283e-01 -1.68721512e+00 -3.01347975e+00 -5.29710333e+00]
 [-9.28953958e-01 -1.74533402e+00 -3.04698389e+00 -5.23345443e+00]
 [-9.64612704e-01 -1.78587561e+00 -3.05587637e+00 -5.14123355e+00]
 [-9.92465497e-01 -1.81408492e+00 -3.04872892e+00 -5.03324231e+00]
 [-1.01435996e+00 -1.83319287e+00 -3.03061351e+00 -4.91658539e+00]
 [-1.03154546e+00 -1.84533280e+00 -3.00474892e+00 -4.79547195e+00]
 [-1.04490992e+00 -1.85198739e+00 -2.97328535e+00 -4.67250965e+00]
 [-1.05510906e+00 -1.85422858e+00 -2.93771645e+00 -4.54936371e+00]
 [-1.06264206e+00 -1.85285607e+00 -2.89911208e+00 -4.42711789e+00]
 [-1.06789855e+00 -1.84848194e+00 -2.85825771e+00 -4.30648377e+00]
 [-1.07118893e+00 -1.84158468e+00 -2.81574162e+00 -4.18792776e+00]
 [